# ZBasePromptTemplate

The `ZBasePromptTemplate` extends `BasePromptTemplate` in multiple ways:

- It allows for partial variables of arbitrary types
- It allows for only overriding `_prompt_type` and `format_prompt` during subclassing, as `format` is rather redundant now.
- `ZChatPromptTemplate` allows for partials, unlike `ChatPromptTemplate` (as of langchain v0.0.125)

It can either be wrapped around an existing `BasePromptTemplate`:

In [1]:
from langchain_contrib.prompts import ZBasePromptTemplate
from langchain.prompts.prompt import PromptTemplate

base = PromptTemplate.from_template("a={a} b={b} c={c}").partial(a="one")
z_base = ZBasePromptTemplate.from_base_template(base).permissive_partial(b=2)
z_base.format(c=[3])

'a=one b=2 c=[3]'

Or it can be subclassed. Note that you no longer need to also override `format` here.

In [2]:
from typing import List, Any
from langchain.prompts.base import StringPromptValue
from langchain.schema import PromptValue


class DemoPromptTemplate(ZBasePromptTemplate):
    """Demonstration of subclassing a ZBasePromptTemplate."""

    input_variables: List[str] = ["a", "b", "c"]

    @property
    def _prompt_type(self) -> str:
        return "demo"

    def _format_prompt(self, **kwargs: Any) -> PromptValue:
        """Return a demonstration of a partial prompt."""
        return StringPromptValue(text="a={a} b={b} c={c}".format(**kwargs))

    
z_base = DemoPromptTemplate().permissive_partial(a="one", b=2)
z_base.format(c=[3])

'a=one b=2 c=[3]'

There also exists `ZStringPromptTemplate`, `ZPromptTemplate`, and `ZChatPromptTemplate`, which should be plug-and-play replacements for their regular Langchain counterparts. `ZChatPromptTemplate` in particular allows for partials of both the regular and the permissive kinds:

In [3]:
from langchain_contrib.prompts import ZChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate

template = ZChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template("a={a} b={b} c={c}")]
)
partial = template.partial(a="one").permissive_partial(b=2)
partial.format_prompt(c=[3])

ChatPromptValue(messages=[SystemMessage(content='a=one b=2 c=[3]', additional_kwargs={})])

The permissive partials also call any functions you pass in, just as the regular kinds do. To reproduce an example from the Langchain docs:

In [4]:
from datetime import datetime
from langchain_contrib.prompts import ZPromptTemplate

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = ZPromptTemplate.from_template(
    "Tell me a {adjective} joke about the day {date}"
)
partial_prompt = prompt.permissive_partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 03/30/2023, 19:59:06
